# Visualization of estimated DNA movement

In this notebook, we load the h5 file containing the results and generate visualization.

In [1]:
%load_ext autoreload
%autoreload 2

Load the results from the h5 file

In [38]:
from pathlib import Path
import motionquant as mq
import matplotlib.pyplot as plt

# define the path to the result folder
dst = Path('./')
# define the result file
filename = dst / "results.h5"
# check the content of the file
items = mq.inspect_result(filename)
print(f"There are {len(items)} items in '{filename}'.")

There are 62 items in 'results.h5'.


Make a figure for a selected a sequences.

In [54]:
plt.rc('font', size=6)
mq.figure(filename, items[10], frame="auto")
#plt.suptitle(items[3], fontsize=8)

Save all figures in a multipage pdf

In [55]:
from matplotlib.backends.backend_pdf import PdfPages

plt.rc('font', size=6)
with PdfPages('figures.pdf') as pdf:
    for name in items:
        plt.clf()
        mq.figure(filename, name, frame="auto")
        plt.suptitle(name,fontsize=8)
        pdf.savefig()
        plt.close()

<Figure size 640x480 with 0 Axes>

Inspect a single strip

In [85]:
name = items[10]
mq.strip(filename, name, "Greys", selection=slice(0,200,20), quiver=False)


Save all strips as individual pdf files

In [86]:
from matplotlib.backends.backend_pdf import PdfPages

plt.rc('font', size=6)
with PdfPages('strip.pdf') as pdf:
    for name in items:
        plt.clf()
        mq.strip(filename, name, "Greys", selection=slice(0,200,20), quiver=False)
        plt.suptitle(name,fontsize=8)
        pdf.savefig()
        plt.close()

<Figure size 640x480 with 0 Axes>

Visualize the image and mask using napari

In [ ]:
import napari
vector = mq.make_vector(rho,4)
v = napari.view_image(img, channel_axis=1)
v.add_labels(mask.squeeze())
v.add_vectors(vector, edge_width=0.1, edge_color="white", length=1)

### Massage the csv file

In [88]:
import pandas as pd
df = pd.read_csv('results.csv',index_col=0)

Add a synchronized time

In [89]:
ndf = []
for k, ng in enumerate(df.groupby('filename')):
    name, group = ng
    t0 = mq.split_frame(group)    
    group['sync frame'] = group['frame'] - t0
    ndf.append(group)
df = pd.concat(ndf)

In [87]:
Add the condition

SyntaxError: invalid syntax (3423932855.py, line 1)

In [103]:
filelist

Unnamed: 0                                               path  \
0            0  /media/cephfs2/jparham/Joe for Jerome /SegA Li...   
1            1  /media/cephfs2/jparham/Joe for Jerome /SegA Li...   
2            2  /media/cephfs2/jparham/Joe for Jerome /SegA Li...   
3            3  /media/cephfs2/jparham/Joe for Jerome /SegA Li...   
4            4  /media/cephfs2/jparham/Joe for Jerome /SegA Li...   
..         ...                                                ...   
57          57  /media/cephfs2/jparham/Joe for Jerome /Divisio...   
58          58  /media/cephfs2/jparham/Joe for Jerome /Divisio...   
59          59  /media/cephfs2/jparham/Joe for Jerome /Divisio...   
60          60  /media/cephfs2/jparham/Joe for Jerome /Divisio...   
61          61  /media/cephfs2/jparham/Joe for Jerome /Divisio...   

                                  name condition  \
0    01a_delSegA_CM_SyS_reg_Cell_0.tif      sega   
1   01a_delSegA_CM_SyS_reg_Cell_12.tif      sega   
2   01a_delSegA_CM_SyS_reg_Cell_13.tif      sega   
3    01a_delSegA_CM_SyS_reg_Cell_3.tif      sega   
4   01a_delSegA_CM_SyS_reg_Cell_11.tif      sega   
..                                 ...       ...   
57        01a_DSM639_SyS_CM_Cell_9.tif        wt   
58        01a_DSM639_SyS_CM_Cell_7.tif        wt   
59        01a_DSM639_SyS_CM_Cell_8.tif        wt   
60    01a_DSM639_SyS_CM_reg_Cell_0.tif        wt   
61       01a_DSM639_SyS_CM_Cell_24.tif        wt   

                          filename  
0    01a_delSegA_CM_SyS_reg_Cell_0  
1   01a_delSegA_CM_SyS_reg_Cell_12  
2   01a_delSegA_CM_SyS_reg_Cell_13  
3    01a_delSegA_CM_SyS_reg_Cell_3  
4   01a_delSegA_CM_SyS_reg_Cell_11  
..                             ...  
57        01a_DSM639_SyS_CM_Cell_9  
58        01a_DSM639_SyS_CM_Cell_7  
59        01a_DSM639_SyS_CM_Cell_8  
60    01a_DSM639_SyS_CM_reg_Cell_0  
61       01a_DSM639_SyS_CM_Cell_24  

[62 rows x 5 columns]

In [111]:
filelist = pd.read_csv('filelist.csv',usecols=['name','condition'])
filelist['filename'] = filelist['name'].apply(lambda x:Path(x).stem)
dfc = df.merge(filelist.drop('name',axis=1),on='filename',how='left')
dfc.to_csv('results_with_condition.csv')

Save all results to a excel file

In [107]:
with pd.ExcelWriter('sheets.xlsx') as writer:
    for tab in df.columns[6:]:
        tmp = df.pivot_table(values=tab, index='frame', columns='filename')
        tmp.to_excel(writer,sheet_name=tab)

Display each measurement by condition

In [126]:
import seaborn as sns
import numpy as np
cols = dfc.columns[5:-2]
fig,ax = plt.subplots(4, int(np.ceil(len(cols)/4)))
ax = ax.ravel()
for k,c in enumerate(cols):
    print(c)
    p = sns.lineplot(data=dfc[dfc["sync frame"]<100], x='sync frame', y=c, hue='condition', ax=ax[k])
    p.get_legend().remove()
fig.set_tight_layout(True)

frame difference
flow
momentum
divergence
dna intensity mean
dna intensity spread
dna intensity skew
dna blob count
dna blob area
dna blob 1 sum intensity
dna blob 2 sum intensity
dna blob 1 area
dna blob 2 area


Use seaborn to create a figure

In [ ]:
dflong = pd.melt(df.iloc[0:1200], id_vars=('filename','frame') , value_vars=df.columns[6:])
dflong.head()

In [ ]:

grid = sns.FacetGrid(data=dflong, row='filename', col='variable', sharey = False)
grid.map_dataframe(sns.lineplot, x="frame",y="value")
# grid.set_titles(template="", row_template='{row_var}{row_name}',col_template='{col_var}{col_name}')
grid.set_titles("", col_template='{col_name}')

In [ ]:
facet_kws={'sharey': False, 'sharex': True}
sns.relplot(data=dflong, x="frame", y="value", col="filename", row="variable", kind='line', facet_kws=facet_kws)